In [13]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import atexit
from time import time, strftime, localtime
from datetime import timedelta

def secondsToStr(elapsed=None):
    if elapsed is None:
        return strftime("%S", localtime())
    else:
        return str(timedelta(seconds=elapsed))

def log(s, elapsed=None):
    line = "="*40
    if elapsed:
        print(elapsed)

def endlog():
    end = time()
    elapsed = end-start
    return secondsToStr(elapsed)

# lấy dữ liệu từ excel
df_course = pd.read_excel(r'C:\Users\DUONG\Desktop\data-recom-small.xlsx', sheet_name = 'Courses')
df_lo = pd.read_excel(r'C:\Users\DUONG\Desktop\data-recom-small.xlsx', sheet_name = 'Learning_outcomes')
df_job = pd.read_excel(r'C:\Users\DUONG\Desktop\data-recom-small.xlsx', sheet_name = 'Path_Jobs')

S = [i for i in range (1,len(df_lo['name'])+1)]
course = [i for i in range (1,len(df_course['name'])+1)]

course_rating = df_course['rate'].copy()
course_number_enroll = df_course['number_enrollment'].copy()
# course_price = [random.randint(50,100) for i in range(len(course_rating))]
course_price = df_course['price'].copy()
course_duration = df_course['duration'].copy()

knowledge_graph = [[0 for i in range(len(S))] for i in range(len(S))]
course_achi = [[0 for i in range(len(S))] for i in range(len(course))]

checknan = df_lo['require'].isnull()

for i in range(len(df_lo['name'])):
    x = df_lo['require'][i]
    if not checknan[i]:
        val = x.split(';')
        for j in val:
            if j != '':
                knowledge_graph[i][int(j)-1] = 1
                
for i in range(len(df_course['name'])):
    x = df_course['LOs'][i]
    val = x.split(';')
    for j in val:
        if j != '':
            course_achi[i][int(j)-1] = 1            

job_applied = [0]
            
# cột mốc
milestone = [0] * len(S)
for i in job_applied: 
    path = df_job['path'][i]
    x = path.split(';')
    for i in x:
        milestone[int(i)-1] = 1
# 7 kĩ năng
inputM = [0] * len(S)

achiM = []
for i,j in zip (inputM,milestone):
    achiM.append(1 if i == 0 and j == 1 else 0)
    
B = 100000
Wf = [1,1,1,1,1]

start = time()
atexit.register(endlog)
log("")

E1 = 0
E2 = 0
E3 = sum(course_rating) / len(course)
E4 = sum(course_number_enroll) / len(course)
E5 = 0


def LOinCourse(courses):
    c = []
    for i in courses: 
        path = df_course['LOs'][i-1]
        x = path.split(';')
        for j in x:
            if j == '':
                continue
            if j not in c:
                c.append(j)
            require_path = df_lo['require'][int(j)-1]
            if not checknan[int(j)-1]:
                val = require_path.split(';')
                for k in val:
                    if k != '':
                        if k not in c:
                            c.append(k)
    return c

def checkReverse(cur,tar):
    t = []
    for i,j in zip(cur,tar):
        t.append(i-j)
    return t

def updateScoreNeeded(cur):
    t = []
    for i,j in zip(milestone,cur):
        t.append(1 if i == 1 and j == 0 else 0)
    return t

def updateCurrentScroce(cur,nex):
    t = []
    for i,j in zip(cur,nex):
        t.append(0 if i == 0 and j == 0 else 1)
    return t

def CanAttendCourse(cur,nex):
    for i,j in zip(cur,nex):
        if j - i == 1:
            return False
    return True

class Choromosome(object):
    def __init__(self,genes,one_row):
        self.one_row = one_row.copy()
        self.genes = genes.copy()
        self.fitness = self.cal_fitness()
    
    def cal_fitness(self):
        
        f1 = 0; f2 = 0; f3 = 0; f4 = 0; f5 = 0;
        gene_copy = self.genes.copy()
        f1 = Choromosome.getLearningPathF1(gene_copy)
        
        f2 = Choromosome.getDurationF2(self.one_row)
        
        f3 = Choromosome.getRatingF3(self.one_row)
        
        f4 = Choromosome.getEnrollNumberF4(self.one_row)
        
        f5 = Choromosome.getCostF5(self.one_row)

        return Choromosome.calcualateFitness(f1,f2,f3,f4,f5)
    
    def calcualateFitness(f1,f2,f3,f4,f5):
        return math.sqrt(math.pow(E1-f1,2) * Wf[0] + math.pow(E2-f2,2) * Wf[1] + math.pow(E3-f3,2) * Wf[2] + math.pow(E4-f4,2) * Wf[3] + math.pow(E5-f5,2) * Wf[4])
    
    def getDurationF2(cur):
        F2 = 0 
        for i in cur:
            F2 += course_duration[i-1]
        return F2

    def getRatingF3(cur):
        F3 = 0 
        for i in cur:
            F3 += course_rating[i-1]
        return F3
    
    def getEnrollNumberF4(cur):
        F4 = 0 
        for i in cur:
            F4 += course_number_enroll[i-1]
        return F4
    
    def getCostF5(cur):
        F5 = 0 
        for i in cur:
            F5 += course_price[i-1]
        return F5
    
    def getLearningPathF1(abc):
        self = abc.copy()
        lst = []
        lastTime = 0
        DEG = [0] * 10005
        for i in range(0,len(course)+1):
            for j in range(0,len(course)):
                if self[i][j] == 1:
                    DEG[j+1] += 1
        dEnd = [0] * 10005
        dEnd[0] = 0
        for i in range(1,len(course)+1):
            dEnd[i] =  1e9
        lst.append(0)
        while len(lst) > 0:
            x = lst[0]
            lst.remove(x)
            curEnd = 0
            for i in range(0,len(course)):
                if self[x][i] == 1:
                    DEG[i+1] -= 1
                    if DEG[i+1] == 0:
                        lst.append(i+1)
            if x > 0:
                for i in range(0,len(course)+1):
                    if self[i][x-1] == 1:
                        curEnd = max(curEnd,dEnd[i]+course_duration[x-1])
            dEnd[x] = curEnd
            lastTime = max(lastTime,dEnd[x])
        return lastTime

def isValidChroVer2(self):
    input_temp = inputM.copy()
    achiM_copy = achiM.copy()
    for i in self:
        if checkLearningOutcomeDepenWithCourse(course_achi[i-1],input_temp):
            input_temp = updateCurrentScroce(input_temp,course_achi[i-1])
            achiM_copy = updateScoreNeeded(input_temp)
            nb = len([i for i in achiM_copy if i == 1])
            if nb == 0:
                return True
        else:
            return False
    return False

def checkLearningOutcomeDepen(index_outcome,curr_achi):
    for i in range(0,len(knowledge_graph[index_outcome])):
        if knowledge_graph[index_outcome][i] == 1 and curr_achi[i] == 0:
            return False
    return True

def checkLearningOutcomeDepenBusiness(course_require,index_outcome,curr_achi):
    for i in range(0,len(knowledge_graph[index_outcome])):
        if knowledge_graph[index_outcome][i] == 1 and curr_achi[i] == 0 and course_require[i] == 0:
            return False
    return True

def checkLearningOutcomeDepenWithCourse(course_require,curr_achi):
    for i in range(0,len(course_require)):
        if course_require[i] == 1 and not checkLearningOutcomeDepenBusiness(course_require,i,curr_achi):
            return False
    return True


def convertToTree(self):
    genes = [[0 for i in range(len(course[::]))] for i in range(len(course[::])+1)]
    temp_curIn = inputM.copy()
    curRoot = inputM.copy()
    course_choose_1_flow = self.copy()
    for i in course_choose_1_flow:
        if checkLearningOutcomeDepenWithCourse(course_achi[i-1],temp_curIn):
            genes[0][i-1] = 1
            curRoot = updateCurrentScroce(curRoot,course_achi[i-1])
    for i in range(1,len(course_choose_1_flow)):
        temp_curIn = curRoot.copy()
        if genes[0][course_choose_1_flow[i]-1] == 0:
            for j in range(0,i):
                temp_curIn = updateCurrentScroce(temp_curIn,course_achi[course_choose_1_flow[j]-1])
                if checkLearningOutcomeDepenWithCourse(course_achi[course_choose_1_flow[i]-1],temp_curIn):
                    genes[course_choose_1_flow[j]][course_choose_1_flow[i]-1] = 1
                    cur_reverse = temp_curIn.copy()
                    for k in range(0,j):
                        if 1 not in genes[course_choose_1_flow[j-k-1]] and genes[0][course_choose_1_flow[k]-1] != 1:
                            cur_reverse = checkReverse(cur_reverse,course_achi[course_choose_1_flow[j-k-1]-1])
                            if checkLearningOutcomeDepenWithCourse(course_achi[course_choose_1_flow[i]-1],cur_reverse):
                                genes[course_choose_1_flow[j-k-1]][course_choose_1_flow[i]-1] = 1
                    break
    return genes
    

def main():
    population = []
    b = []
    for i in course:
        c = []
        if i not in c:
            c.append(i)
        if c not in b:
            b.append(c.copy())
        for j in course:
            c1 = c.copy()
            if j not in c1:
                c1.append(j)
            if c1 not in b:
                b.append(c1.copy())
            for k in course:
                c2 = c1.copy()
                if k not in c2:
                    c2.append(k)
                if c2 not in b:
                    b.append(c2.copy())
                for h in course:
                    c3 = c2.copy()
                    if h not in c3:
                        c3.append(h)
                    if c3 not in b:
                        b.append(c3.copy())
                    for g in course:
                        c4 = c3.copy()
                        if g not in c4:
                            c4.append(g)
                        if c4 not in b:
                            b.append(c4.copy())
                        for f in course:
                            c5 = c4.copy()
                            if f not in c5:
                                c5.append(f)
                            if c5 not in b:
                                b.append(c5.copy())
                            for v in course:
                                c6 = c5.copy()
                                if v not in c6:
                                    c6.append(v)
                                if c6 not in b:
                                    b.append(c6.copy())
                                for d in course:
                                    c7 = c6.copy()
                                    if d not in c7:
                                        c6.append(d)
                                    if c7 not in b:
                                        b.append(c7.copy())
    for i in b:
        if isValidChroVer2(i):
            genes = convertToTree(i)
            population.append(Choromosome(genes.copy(),i.copy()))
    population = sorted(population, key = lambda x:x.fitness)
    print("Path: {0}\tFitness: {1}".
    format(population[0].one_row,population[0].fitness))
    
    ff1 = Choromosome.getLearningPathF1(population[0].genes)
    ff2 = Choromosome.getDurationF2(population[0].one_row)
    ff3 = Choromosome.getRatingF3(population[0].one_row)
    ff4 = Choromosome.getEnrollNumberF4(population[0].one_row)
    ff5 = Choromosome.getCostF5(population[0].one_row)
    
    print("{0}\t{1}\t{2}\t{3}\t{4}".
        format(ff1,ff2,ff3,ff4,ff5))
if __name__ == '__main__':
    main()
print("\n\nExcution time: {0}s".format(endlog()))

Path: [5, 2, 7]	Fitness: 124601.9663223533
48.0	56.76667	13.100000000000001	247523	147


Excution time: 0:12:45.332891s


In [10]:
print("{0}".format([1,2,3]))

[1, 2, 3]
